In [1]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns
import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
goog = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/Finance/main/dataset/Silicon%20Valley%20Stock%20Prices%20(Apple%2C%20Google%20%26%20more)/GOOG.csv")
amzn=pd.read_csv('https://raw.githubusercontent.com/benvictoria17/Finance/main/dataset/Silicon%20Valley%20Stock%20Prices%20(Apple%2C%20Google%20%26%20more)/AMZN.csv')
nflx=pd.read_csv('https://raw.githubusercontent.com/benvictoria17/Finance/main/dataset/Silicon%20Valley%20Stock%20Prices%20(Apple%2C%20Google%20%26%20more)/NFLX.csv')

In [3]:
data=pd.DataFrame()
data['Date']=goog['Date']
data['Google']=goog['Open']
data['Netflix']=nflx['Open']
data['Amazon']=amzn['Open']
data

,Date,Google,Netflix,Amazon
0,2004-08-20,50.316402,1.214286,1.968750
1,2004-08-23,55.168217,1.213571,1.760417
2,2004-08-24,55.412300,1.164286,1.729167
3,2004-08-25,52.284027,1.107857,1.635417
4,2004-08-26,52.279045,1.078571,1.437500
...,...,...,...,...
4275,2021-08-13,2767.149902,351.230011,302.500000
4276,2021-08-16,2760.000000,350.950012,298.019989
4277,2021-08-17,2763.820068,358.010010,292.799988
4278,2021-08-18,2742.310059,355.500000,295.760010


In [4]:
data['Date']=pd.to_datetime(data['Date'])
data=data.sort_values('Date').reset_index(drop=True)
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data['Date'],y=data['Google'],name='Google'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data['Date'],y=data['Netflix'],name='Netflix'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data['Date'],y=data['Amazon'],name='Amazon'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Google, Amazon, and Netflix Stocks")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price",secondary_y=False)
fig.show()
col0=data.columns.to_list()
col1=col0+['Google 7d-before','slope']
data1=pd.DataFrame(columns=col1)
data1[col0]=data
data1

,Date,Google,Netflix,Amazon,Google 7d-before,slope
0,2004-08-20,50.316402,1.214286,1.968750,NaN,NaN
1,2004-08-23,55.168217,1.213571,1.760417,NaN,NaN
2,2004-08-24,55.412300,1.164286,1.729167,NaN,NaN
3,2004-08-25,52.284027,1.107857,1.635417,NaN,NaN
4,2004-08-26,52.279045,1.078571,1.437500,NaN,NaN
...,...,...,...,...,...,...
4275,2021-08-13,2767.149902,351.230011,302.500000,NaN,NaN
4276,2021-08-16,2760.000000,350.950012,298.019989,NaN,NaN
4277,2021-08-17,2763.820068,358.010010,292.799988,NaN,NaN
4278,2021-08-18,2742.310059,355.500000,295.760010,NaN,NaN


In [5]:
n=len(data)
for  i in range(n):    
    pmi=data1['Google'][i]
    data1.loc[i+7,'Google 7d-before']=pmi
data1

,Date,Google,Netflix,Amazon,Google 7d-before,slope
0,2004-08-20,50.316402,1.214286,1.968750,NaN,NaN
1,2004-08-23,55.168217,1.213571,1.760417,NaN,NaN
2,2004-08-24,55.412300,1.164286,1.729167,NaN,NaN
3,2004-08-25,52.284027,1.107857,1.635417,NaN,NaN
4,2004-08-26,52.279045,1.078571,1.437500,NaN,NaN
...,...,...,...,...,...,...
4282,NaT,NaN,NaN,NaN,2767.149902,NaN
4283,NaT,NaN,NaN,NaN,2760.0,NaN
4284,NaT,NaN,NaN,NaN,2763.820068,NaN
4285,NaT,NaN,NaN,NaN,2742.310059,NaN


In [6]:
data1['slope']=(data1['Google']-data1['Google 7d-before'])/7
data2=data1[['Date','Google','Google 7d-before','slope']]
data3=data2[14:-7]

data3

,Date,Google,Google 7d-before,slope
14,2004-09-10,50.610298,50.958992,-0.049813
15,2004-09-13,53.115910,51.158245,0.279666
16,2004-09-14,53.524376,49.409801,0.587796
17,2004-09-15,55.073570,50.286514,0.683865
18,2004-09-16,55.960247,50.316402,0.806264
...,...,...,...,...
4275,2021-08-13,2767.149902,2724.98999,6.022845
4276,2021-08-16,2760.000000,2720.570068,5.632847
4277,2021-08-17,2763.820068,2725.899902,5.417167
4278,2021-08-18,2742.310059,2738.97998,0.475726


In [7]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data3['Date'],y=data3['slope'],name='slope'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Google Stocks Slope change")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="Slope change",secondary_y=True)
fig.show()
data3[data3['slope']<-25]

,Date,Google,Google 7d-before,slope
3907,2020-02-28,1277.500000,1525.069946,-35.367135
3916,2020-03-12,1126.000000,1399.420044,-39.060006
3917,2020-03-13,1179.000000,1359.22998,-25.74714
3918,2020-03-16,1096.000000,1350.199951,-36.314279
3919,2020-03-17,1093.109985,1277.060059,-26.278582
